In [1]:
import os
import re
import glob
import logging
import pandas as pd
from pathlib import Path

from groq import Groq
from tqdm import tqdm
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)
logging.basicConfig(filename='knowledge_crystallizer_errors.log', level=logging.ERROR, encoding='utf-8')

In [3]:
# input_files_path = '../data/raw/library/psychology/es/'
# output_files_path = '../data/clean/knowledge/psychology/es/'

input_files_path = '../data/raw/news/'
output_files_path = '../data/clean/news/'

os.makedirs(input_files_path, exist_ok=True)
os.makedirs(output_files_path, exist_ok=True)

In [4]:
def extract_markdown_sections(markdown_text):
    # Split the text into lines
    lines = markdown_text.strip().split('\n')
    
    # Initialize variables
    current_headers = []
    sections = []
    current_content = []
    
    def get_header_level(line):
        """Count the number of # at the start of the line"""
        match = re.match(r'^(#+)\s', line)
        return len(match.group(1)) if match else 0
    
    def save_current_section():
        """Save the current section if there's any content"""
        if current_content:
            # Combine all current headers and content
            section = '\n'.join(current_headers + [''] + current_content).strip()
            sections.append(section)
            current_content.clear()
    
    for line in lines:
        level = get_header_level(line)
        
        if level > 0:  # This is a header line
            # Save the previous section if it exists
            save_current_section()
            
            # Update the current headers based on the level
            while len(current_headers) >= level:
                current_headers.pop()
            current_headers.append(line)
        
        elif line.strip():  # This is a content line
            current_content.append(line)
    
    # Save the last section
    save_current_section()
    
    return sections

In [5]:
def exctract_knowledge(content):
    template = f'''Extrae TODO el conocimiento del texto de entrada como una colección de hechos completamente independientes y autocontenidos. Cada hecho debe contener todo lo necesario para comprenderlo sin necesidad de hacer referencia a otros hechos o al contexto externo

**Conserve los nombres propios SÓLO cuando sean**:
- Autores de descubrimientos o teorías científicas verificadas
- Nombres de leyes, principios o teorías establecidas
- Patentes o innovaciones tecnológicas documentadas
- Acontecimientos históricos en los que la fecha o el lugar concretos son cruciales
- Lugares geográficos cuando sus propiedades específicas sean relevantes para el hecho
- Medidas científicas o normas establecidas

**Instrucciones para la extracción de cada hecho**:
- Utiliza términos genéricos en lugar de nombres propios no esenciales (por ejemplo, «mamíferos» en lugar de nombres específicos de animales)
- Sustituya todos los términos relativos (como «muchos», «algunos», «a menudo») por cantidades concretas
- Convierta las afirmaciones generales en afirmaciones precisas y mensurables
- Incluya condiciones y contextos específicos
- Utilice terminología exacta en lugar de aproximaciones
- Incluya todo el contexto necesario dentro del mismo párrafo
- Indique todas las condiciones y cualificaciones pertinentes
- Defina los términos especializados dentro del mismo hecho
- Incluya todos los detalles cruciales necesarios para la comprensión
- Incluya sólo la información esencial para el hecho concreto
- Elimine el lenguaje decorativo y los detalles innecesarios
- Céntrese en un punto claro por hecho
- Excluya la información tangencial
- Utilice un lenguaje claro y literal
- Evite metáforas y modismos
- Defina los términos potencialmente ambiguos
- Indique explícitamente las relaciones
- Utilice la misma terminología en todos los hechos
- Mantenga un nivel de detalle coherente
- Asegúrese de que los hechos complementarios no se contradigan
- Ajustese al nivel técnico del material original
- Proporcione detalles suficientes para la comprensión práctica
- Incluya toda la información contextual pertinente

**Formato de salida**:

Tu resultado final debe presentar sólo y unicamente una simple lista numerada de hechos, por ejemplo:

1. [Primer hecho completo y autónomo en un único párrafo].
2. [Segundo hecho completo y autónomo en un único párrafo].
3. [Y así sucesivamente...].

**Cada hecho debe**:
- Ser completamente comprensible por sí solo
- Contener todo el contexto y las definiciones necesarias
- Ser preciso e inequívoco
- Incluir sólo la información pertinente
- Expresarse en un lenguaje claro y literal.

**La lista completa de hechos debe conservar todos los conocimientos del texto original sin pérdida de información ni de contexto**

# Texto de entrada:

"""
{content}
"""'''

    client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": template,
            }
        ],
        model=os.environ.get("GROQ_MODEL_ID"),
        max_tokens=8000,
        top_p=0.1,
        temperature=0.92
    )

    return chat_completion.choices[0].message.content

In [6]:
def facts_to_dataframe(text):
    pattern = r"^\s*(\d+)\.\s+(.+?)(?=\n\s*\d+\.|$)"
    matches = re.finditer(pattern, text, re.MULTILINE | re.DOTALL)
    facts = []
    for match in matches:
        fact = match.group(2).strip()
        facts.append(fact)
    df = pd.DataFrame({"fact": facts})
    return df

In [7]:
for input_filename in glob.glob(input_files_path + '*.md'):
    file_facts = []
    output_filename = f'{output_files_path}{Path(input_filename).stem}.csv'

    with open(input_filename,'r', encoding='utf-8') as f:
        book = f.read()  
        sections = extract_markdown_sections(book)

        for section in tqdm(sections, total=len(sections), desc=f"Extracting knowledge from {input_filename}..."):
            try:
                facts = exctract_knowledge(section)
                file_facts.append(facts)
            except Exception as e:
                error_log = f"""## Section:
{section}
## Error:
{str(e)}
-----------------------------------------------
-----------------------------------------------"""
                logging.error(error_log)

        all_file_facts_text = '\n\n'.join(file_facts)
        df = facts_to_dataframe(all_file_facts_text)
        df.to_csv(output_filename, index=False, encoding='utf-8')

In [10]:
for input_filename in glob.glob(input_files_path + '*.txt'):
    file_facts = []
    output_filename = f'{output_files_path}{Path(input_filename).stem}.csv'

    with open(input_filename,'r', encoding='utf-8') as f:
        content = f.read()  
        sections = [content[i:i + 4096] for i in range(0, len(content), 4096)]

        for section in tqdm(sections, total=len(sections), desc=f"Extracting knowledge from {input_filename}..."):
            try:
                facts = exctract_knowledge(section)
                file_facts.append(facts)
            except Exception as e:
                error_log = f"""## Section:
{section}
## Error:
{str(e)}
-----------------------------------------------
-----------------------------------------------"""
                logging.error(error_log)

        all_file_facts_text = '\n\n'.join(file_facts)
        df = facts_to_dataframe(all_file_facts_text)
        df.to_csv(output_filename, index=False, encoding='utf-8')

Extracting knowledge from ../data/raw/news\8_GameChanging_Manufacturing_Trends_That_Will_Define_2025.txt...: 100%|██████████| 2/2 [00:05<00:00,  2.90s/it]
Extracting knowledge from ../data/raw/news\At_AWS_ReInvent_A_Look_At_Reinventing_AI.txt...: 100%|██████████| 3/3 [00:23<00:00,  7.71s/it]
Extracting knowledge from ../data/raw/news\Deep_Dive_AI_2024.txt...: 100%|██████████| 1/1 [00:17<00:00, 17.66s/it]
Extracting knowledge from ../data/raw/news\How_AI_can_help_you_attract_engage_and_retain_the_best_talent_in_2025.txt...: 100%|██████████| 2/2 [00:32<00:00, 16.39s/it]
Extracting knowledge from ../data/raw/news\Lately_The_years_biggest_YouTube_trends_AI_weather_forecasts_and_brain_rot.txt...: 100%|██████████| 2/2 [00:36<00:00, 18.22s/it]


## Vector Index

In [8]:
# dfs = []
# for fact_filename in glob.glob(output_files_path + '*.csv'):
#     df = pd.read_csv(fact_filename, encoding='utf-8')
#     dfs.append(df)
# all_facts_df = pd.concat(dfs, ignore_index=True)

In [9]:
# all_facts_df['fact'].str.split().str.len()

In [ ]:
# for input_filename in glob.glob(input_files_path + '*.txt'):
#     file_facts = []
#     output_filename = f'{output_files_path}{Path(input_filename).stem}.csv'

#     with open(input_filename,'r', encoding='utf-8') as f:
#         content = f.read() 
#         print(len(content))

6781
8795
3175
5253
5674
